In [1]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

!pip install wandb --quiet
!pip install python-Levenshtein -q

!pip install torchsummaryX -q
!pip install mne -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 744.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 30.8 MB/s eta 0:00:00


In [2]:
import torch
import torchaudio
from torch import nn, Tensor
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# import torchsummary

import numpy as np
import os

import gc
import time

import pandas as pd
from tqdm.notebook import tqdm as blue_tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import json

import math
from typing import Optional, List


#imports for decoding and distance calculation
try:
    import wandb
    import torchsummaryX
    import Levenshtein
except:
    print("Didnt install some/all imports")

import sklearn
import mne

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
DEVICE=device
print("Device: ", device)

Device:  cuda


In [3]:
config = dict (
    batch_size          = 36,
    epochs              = 100,
    learning_rate       = 1e-4, # per write up
    weight_decay        = 0,
)

# Suggested transforms per the write up
transforms = torch.nn.Sequential(
    torchaudio.transforms.TimeMasking(time_mask_param=225),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=6)
)

In [4]:
VOCAB = [
    '<pad>', '<sos>', '<eos>',
    'A',   'B',    'C',    'D',
    'E',   'F',    'G',    'H',
    'I',   'J',    'K',    'L',
    'M',   'N',    'O',    'P',
    'Q',   'R',    'S',    'T',
    'U',   'V',    'W',    'X',
    'Y',   'Z',    "'",    ' ',
]

VOCAB_MAP = {VOCAB[i]:i for i in range(0, len(VOCAB))}

PAD_TOKEN = VOCAB_MAP["<pad>"]
SOS_TOKEN = VOCAB_MAP["<sos>"]
EOS_TOKEN = VOCAB_MAP["<eos>"]

print(f"Length of vocab : {len(VOCAB)}")
print(f"Vocab           : {VOCAB}")
print(f"PAD_TOKEN       : {PAD_TOKEN}")
print(f"SOS_TOKEN       : {SOS_TOKEN}")
print(f"EOS_TOKEN       : {EOS_TOKEN}")

Length of vocab : 31
Vocab           : ['<pad>', '<sos>', '<eos>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', "'", ' ']
PAD_TOKEN       : 0
SOS_TOKEN       : 1
EOS_TOKEN       : 2


In [5]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

Mounted at /content/drive


In [6]:
# Load in the new and glorious Phoneme-added dataframe
import pandas as pd
data_csv =pd.read_csv("drive/MyDrive/brennan data/AliceChapterOne-EEG-Phonemes.csv")
data_csv.head()

,Unnamed: 0,Word,Segment,onset,offset,Order,LogFreq,LogFreq_Prev,LogFreq_Next,SndPower,Length,Position,Sentence,IsLexical,NGRAM,RNN,CFG,phonemes
0,0,Alice,1,0.046000,0.608721,1,8.65,0.00,14.56,3.620000e-07,0.562721,1,1,1.0,3.226499,3.126175,2.312348,"['AE1', 'L', 'IH0', 'S']"
1,1,was,1,0.562721,0.830543,2,14.56,8.65,10.69,3.840000e-09,0.267822,2,1,0.0,0.905229,1.691128,1.357460,"['W', 'AH0', 'Z']"
2,2,beginning,1,0.784543,1.302929,3,10.69,14.56,16.35,3.690000e-09,0.518386,3,1,1.0,4.446766,4.100771,5.626722,"['B', 'IH0', 'G', 'IH1', 'N', 'IH0', 'NG']"
3,3,to,1,1.256929,1.398925,4,16.35,10.69,13.79,3.970000e-09,0.141996,4,1,0.0,2.537495,3.833313,5.939201,"['T', 'AH0']"
4,4,get,1,1.352925,1.662327,5,13.79,16.35,13.28,3.770000e-09,0.309402,5,1,0.0,1.023137,1.013076,2.697304,"['G', 'IH1', 'T']"


In [7]:
TRANSCRIPTS = []

for i in range(len(np.unique(data_csv.Segment))):
  words = " ".join(data_csv["Word"].loc[data_csv.Segment == i+1].values.tolist())
  words = "".join([i.upper() for i in words])
  TRANSCRIPTS.append(words)

print(TRANSCRIPTS)

['ALICE WAS BEGINNING TO GET VERY TIRED OF SITTING BY HER SISTER ON THE BANK AND OF HAVING NOTHING TO DO ONCE OR TWICE SHE PEEPED INTO THE BOOK HER SISTER WAS READING BUT IT HAD NO PICTURES OR CONVERSATIONS IN IT AND WHAT THE USE OF A BOOK THOUGHT ALICE WITHOUT PICTURES OR CONVERSATION SO SHE WAS CONSIDERING IN HER OWN MIND AS WELL AS SHE COULD FOR THE HOT DAY MADE HER FEEL VERY SLEEPY AND STUPID WHETHER THE PLEASURE OF MAKING A DAISY CHAIN WOULD BE WORTH THE TROUBLE OF GETTING UP AND PICKING THE DAISIES WHEN SUDDENLY A WHITE RABBIT WITH PINK EYES RAN CLOSE BY HER THERE WAS NOTHING SO VERY REMARKABLE IN THAT NOR DID ALICE THINK IT SO VERY MUCH OUT OF THE WAY TO HEAR THE RABBIT SAY TO ITSELF OH DEAR OH DEAR I SHALL BE LATE WHEN SHE THOUGHT IT OVER AFTERWARDS IT OCCURRED TO HER THAT SHE OUGHT TO HAVE WONDERED AT THIS BUT AT THE TIME IT ALL SEEMED QUITE NATURAL', 'BUT WHEN THE RABBIT ACTUALLY TOOK A WATCH OUT OF ITS POCKET AND LOOKED AT IT AND THEN HURRIED ON ALICE STARTED TO HER FEET FOR

In [8]:
class EEGSpeechDatasetME(torch.utils.data.Dataset): # Memory efficient
    # Loades the data in get item to save RAM

    def __init__(self, EEG_data, transcripts = TRANSCRIPTS):

        self.VOCAB      = VOCAB

        self.transcripts_files = TRANSCRIPTS

        assert len(EEG_data) == len(transcripts)

        length = len(mfcc_files) # TODO

        self.EEG_files          =  EEG_data
        self.length             = len(self.EEG_files)

    def __len__(self):
        # TODO
        return self.length

    def __getitem__(self, ind):

        # Load the mfcc and transcripts from the mfcc and transcript paths created earlier
        EEG        = self.EEG_files[ind]  # TODO

        transcript  = self.transcript_files[ind] # TODO
        transcript_mapped   = [SOS_TOKEN] + [VOCAB_MAP[i] for i in transcript] + [EOS_TOKEN]# TODO
        transcript_mapped =  np.array(transcript_mapped)

        return torch.FloatTensor(EEG), torch.LongTensor(transcript_mapped)


    def collate_fn(self,batch):

        batch_x, batch_y, lengths_x, lengths_y = [], [], [], []

        for x, y in batch:
            # Add the mfcc, transcripts and their lengths to the lists created above
            # TODO
            batch_x.append(x)
            batch_y.append(y)
            lengths_x.append(len(x))
            lengths_y.append(len(y))

        # pack the mfccs and transcripts using the pad_sequence function from pytorch
        batch_x_pad = pad_sequence(batch_x, batch_first=True, padding_value=PAD_TOKEN)# TODO
        batch_y_pad = pad_sequence(batch_y, batch_first=True, padding_value=PAD_TOKEN)# TODO

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)

In [9]:
import glob
from collections import Counter

vhdr_files = glob.glob("drive/MyDrive/brennan data/*/*.vhdr") # only use five? smaller dataset, more neurons??

In [10]:
  eeg_data_list = []
  mne.set_log_level('WARNING')

  for num, vhdr_file in enumerate(vhdr_files):
    print(f"Loading {num + 1} of {len(vhdr_files)}")

    # Step 1: Load the file
    mne_raw = mne.io.read_raw_brainvision(vhdr_file, misc=['ECG', 'EMG', 'FootPad'],preload=True)

    # A couple of the subjects have a different number of channels.
    # As someone who does EEG data research, I get it.
    # As someone doing this class project, I hate them.
    # Let's just ignore those people with different cap sizes to simplify things
    if mne_raw.get_data().shape[0] == 62:

      # Step 2: Preprocess the filter
      # Note: This is very minimal, but we might not want to do too much more

      # Bandpass filter 0.5 - 40 Hz
      mne_raw = mne_raw.filter(0.5, 40);

      # Common Average Reference
      mne_raw.set_eeg_reference('average', projection=False);

      # Resampling to 100 Hz is done AFTER segmentation into words to not lose
      # time resolution when cropping (see: .resample(100) below)

      # Step 3: Segment into the 12 time-stamped segments
      # For some reason, some of the dataset first stimulus is called "stimulus 1"
      # and other times is new segment -.- . Also, calling events from annotations
      # can return the list in unsorted order, which will completely mess up the
      # whole labels to eeg alignment we have going on.
      # So,
      # mne_events, ev_id = mne.events_from_annotations(
      #           raw=mne_raw, event_id={"Stimulus/{:d}".format(i): i for i in range(0,13)})

      mne_events, ev_id = mne.events_from_annotations(raw=mne_raw, event_id=Counter(mne_raw.annotations.description))
      # because strings are weird, this actually returns the events out of order.
      # so, let's sort them according to their timestamp. Easy peasy!
      mne_events = sorted(mne_events, key = lambda x: x[0])


      if len(mne_events) > 12:
        # Only keep last 12 if there was an extra starting stimulus,
        # Otherwise, consider the starting stimulus as start of sequence
        mne_events = mne_events[-12:]

      # Crop each segment and save it seperately
      segments = [mne_raw.copy().crop(mne_events[i][0] * 1/mne_raw.info['sfreq'], mne_events[i+1][0] * 1/mne_raw.info['sfreq']).resample(100) for i in range(len(mne_events) - 1)]
      segments.append(mne_raw.copy().crop(mne_events[-1][0] * 1/mne_raw.info['sfreq'], None).resample(100))
      eeg_data_list.extend(segments)
      # print(eeg_data_list)

Loading 1 of 49
Loading 2 of 49
Loading 3 of 49
Loading 4 of 49
Loading 5 of 49
Loading 6 of 49
Loading 7 of 49
Loading 8 of 49
Loading 9 of 49
Loading 10 of 49
Loading 11 of 49
Loading 12 of 49
Loading 13 of 49
Loading 14 of 49
Loading 15 of 49
Loading 16 of 49
Loading 17 of 49
Loading 18 of 49
Loading 19 of 49
Loading 20 of 49
Loading 21 of 49
Loading 22 of 49
Loading 23 of 49
Loading 24 of 49
Loading 25 of 49
Loading 26 of 49
Loading 27 of 49
Loading 28 of 49
Loading 29 of 49
Loading 30 of 49
Loading 31 of 49
Loading 32 of 49
Loading 33 of 49
Loading 34 of 49
Loading 35 of 49
Loading 36 of 49
Loading 37 of 49
Loading 38 of 49
Loading 39 of 49
Loading 40 of 49
Loading 41 of 49
Loading 42 of 49
Loading 43 of 49
Loading 44 of 49
Loading 45 of 49
Loading 46 of 49
Loading 47 of 49
Loading 48 of 49
Loading 49 of 49


In [11]:
# DATA_DIR    = "drive/MyDrive/brennan data/S01" # TODO: Path where you have downloaded the data

# mne_raw = mne.io.read_raw_brainvision(os.path.join(DATA_DIR, "S01.vhdr"),
                                      # misc=['ECG', 'EMG', 'FootPad'],preload=True)

In [12]:
# # You'll notice in the dataframe that they've timestamped each word with regard to
# # the segmented file! That was very nice of them. Let's extract the word onset/offset
# import numpy as np
# segment_onset_offset = []
# for seg_val in np.unique(data_csv["Segment"].values):
#   seg = data_csv.loc[data_csv.Segment == seg_val]
#   onsets = seg["onset"].values
#   offsets = seg["offset"].values
#   # Append to onset/offset times
#   segment_onset_offset.append(list(zip(onsets, offsets)))


In [13]:
# # Now its time to load in the EEG data
# # For this, we have to loop through all the subjects (49) and extract
# # each word based on the segment and timestamp
# import mne
# import glob
# from collections import Counter

# if wannaSpendForeverMakingDataAgain:

#   vhdr_files = glob.glob("drive/MyDrive/brennan data/*/*.vhdr")

#   vhdr_files

#   eeg_data_list = []
#   labels_list = []


#   def data_sorter(data_string):
#       try:
#           return int(data_string.split(r"/")[-1])
#       except:
#           # handle division by zero error
#           # leave empty for now
#           return 0

#   for vhdr_file in vhdr_files[26:]:

#     # Step 1: Load the file
#     mne_raw = mne.io.read_raw_brainvision(vhdr_file, misc=['ECG', 'EMG', 'FootPad'],preload=True)


#     # A couple of the subjects have a different number of channels.
#     # As someone who does EEG data research, I get it.
#     # As someone doing this class project, I hate them.
#     # Let's just ignore those people with different cap sizes to simplify things
#     if mne_raw.get_data().shape[0] == 62:

#       # Step 2: Preprocess the filter
#       # Note: This is very minimal, but we might not want to do too much more

#       # Bandpass filter 0.5 - 40 Hz
#       mne_raw = mne_raw.filter(0.5, 40);

#       # Common Average Reference
#       mne_raw.set_eeg_reference('average', projection=False);

#       # Resampling to 100 Hz is done AFTER segmentation into words to not lose
#       # time resolution when cropping (see: .resample(100) below)

#       # Step 3: Segment into the 12 time-stamped segments
#       # For some reason, some of the dataset first stimulus is called "stimulus 1"
#       # and other times is new segment -.- . Also, calling events from annotations
#       # can return the list in unsorted order, which will completely mess up the
#       # whole labels to eeg alignment we have going on.
#       # So,
#       # mne_events, ev_id = mne.events_from_annotations(
#       #           raw=mne_raw, event_id={"Stimulus/{:d}".format(i): i for i in range(0,13)})

#       mne_events, ev_id = mne.events_from_annotations(raw=mne_raw, event_id=Counter(mne_raw.annotations.description))
#       # because strings are weird, this actually returns the events out of order.
#       # so, let's sort them according to their timestamp. Easy peasy!
#       mne_events = sorted(mne_events, key = lambda x: x[0])

#       if len(mne_events) > 12:
#         # Only keep last 12 if there was an extra starting stimulus,
#         # Otherwise, consider the starting stimulus as start of sequence
#         mne_events = mne_events[-12:]

#       # We're just going to crop at onset time. This is because the eeg timestamps sometimes overlap
#       # into the next segment. *eye roll*
#       segments = [mne_raw.copy().crop(mne_events[i][0] * 1/mne_raw.info['sfreq'], None) for i in range(len(mne_events))]

#       # Step 4: Further break down each segment into the individual words
#       for i in range(len(segments)): # there should be 12extracted_sentences = [sentences[0].copy().crop(timestamps[i], timestamps[i+1]) for i in range(len(timestamps) - 1)]
#         #print("Segment: ", i)
#         extracted_words = [segments[i].copy().crop(segment_onset_offset[i][j][0], segment_onset_offset[i][j][1]).resample(100) for j in range(len(segment_onset_offset[i]))]
#         eeg_data_list.extend(extracted_words)
#         #print(eeg_data_list)


In [14]:

#targets = data_csv['phonemes']
#phoneme_list = phoneme_list * len(vhdr_files) # Repeat the labels for each subject

num_subj = len(eeg_data_list) // 12 # 12 sets per subject
targets = []
[targets.extend(TRANSCRIPTS) for _ in range(num_subj)]
print(len(eeg_data_list), len(targets))


564 564


In [15]:
# split into training, testing, and validation set
import sklearn
from sklearn import model_selection

train_targets, val_targets, train_eeg, val_eeg = sklearn.model_selection.train_test_split(targets, eeg_data_list, test_size=0.20, random_state=42) # 80/20 train-val/test split
# train_targets, val_targets, train_eeg, val_eeg = sklearn.model_selection.train_test_split(train_val_targets, train_val_eeg, test_size=0.20, random_state=42) # 80/20 train / val split

In [38]:
# import torch

# class Dataset(torch.utils.data.Dataset):

#     def __init__(self, input_data, # EEG data
#                  output_data = None, # phoneme data
#                  mode = 'Training',
#                  phonemes = PHONEMES,
#                  min_length = 0,
#                 ): # Feel free to add more arguments

#         """
#         Input:
#         input_data: Preprocessed EEG data of array of shape (N_samples, ), where each sample is shape (n_time, n_channels),
#                     where n_channels is a consistent number across all samples
#         output_data: phoneme data of read transcript shape (N_samples, ), where each sample is shape (n_phonemes)

#         """
#         assert mode in ['Training', 'Testing', 'Validation'], "Error! Unrecognized Mode"

#         if mode != "Testing":
#           data_tuple = [(i, j) for i,j in zip(input_data, output_data) if i.shape[0] > min_length]
#           input_data = [i[0] for i in data_tuple]
#           output_data = [i[1] for i in data_tuple]
#           print("Input data len: ", len(input_data))
#           assert len(input_data) == len(output_data)

#         else:
#           input_data = [i for i in input_data if i.shape[0] > min_length]


#         self.EEG = input_data
#         self.length = len(self.EEG)
#         self.mode = mode

#         if mode != 'Testing':
#             self.transcripts = []
#             for transcript in output_data: #transcript is one word
#                 # print(transcript)
#                 transcript_list = []
#                 for phoneme in transcript:
#                   phoneme = ''.join([i for i in phoneme if not i.isdigit()])
#                   transcript_list.append(phonemes.index(phoneme))

#                 self.transcripts.append(transcript_list)
#         #print(self.transcripts)

#     def __len__(self):
#         return self.length

#     def __getitem__(self, ind):

#         frames = self.EEG[int(ind)]
#         frames = torch.FloatTensor(frames) # Convert to tensors

#         if self.mode in ["Training", "Validation"]:
#             #labels = torch.tensor(self.transcripts, dtype=torch.long)
#             labels    = torch.tensor(self.transcripts[ind], dtype=torch.long)
#             return frames, labels

#         else:
#             return frames

#     def collate_fn(self, batch):
#         '''
#         TODO:
#         1.  Extract the features and labels from 'batch'
#         2.  We will additionally need to pad both features and labels,
#             look at pytorch's docs for pad_sequence
#         3.  This is a good place to perform transforms, if you so wish.
#             Performing them on batches will speed the process up a bit.
#         4.  Return batch of features, labels, lenghts of features,
#             and lengths of labels.
#         '''
#         # batch of input mfcc coefficients
#         if self.mode in ["Training", "Validation"]:
#             batch_EEG =  [x[0] for x in batch]
#             lengths_EEG = [len(x[0]) for x in batch]# TODO
#             batch_EEG_pad = pad_sequence(batch_EEG, batch_first=True, padding_value=0.0)

#             #batch_transcript = [x[0][1] for x in batch] # TODO
#             batch_transcript = [x[1] for x in batch]

#             lengths_transcript = [len(x[1]) for x in batch] # TODO
#             #print('lengths_transcript is ', lengths_transcript)
#             #batch_transcript =  torch.tensor(batch_transcript)


#             # Batch x Time x 62 channels
#             batch_transcript_pad = pad_sequence(batch_transcript,
#                                                 batch_first=True,
#                                                 padding_value=0.0) # TODO

#             return batch_EEG_pad, batch_transcript_pad, torch.tensor(lengths_EEG), torch.tensor(lengths_transcript)

#         else:
#             batch_EEG =  [x for x in batch]
#             lengths_EEG = [len(x) for x in batch]# TODO
#             batch_EEG_pad = pad_sequence(batch_EEG, batch_first=True, padding_value=0.0)

#             return batch_EEG_pad, torch.tensor(batch_EEG)

In [39]:
class EEGSpeechDatasetME(torch.utils.data.Dataset): # Memory efficient
    # Loades the data in get item to save RAM

    def __init__(self, EEG_data, transcripts):

        self.VOCAB      = VOCAB
        self.transcript_files = transcripts

        assert len(EEG_data) == len(transcripts)

        self.EEG_files          =  EEG_data
        self.length             =  len(self.EEG_files)

    def __len__(self):
        # TODO
        return self.length

    def __getitem__(self, ind):

        # Load the mfcc and transcripts from the mfcc and transcript paths created earlier
        EEG        = self.EEG_files[ind]  # TODO

        transcript  = self.transcript_files[ind] # TODO
        transcript_mapped   = [SOS_TOKEN] + [VOCAB_MAP[i] for i in transcript] + [EOS_TOKEN] # TODO
        transcript_mapped =  np.array(transcript_mapped)

        return torch.FloatTensor(EEG), torch.LongTensor(transcript_mapped)


    def collate_fn(self,batch):

        batch_x, batch_y, lengths_x, lengths_y = [], [], [], []

        for x, y in batch:
            # Add the mfcc, transcripts and their lengths to the lists created above
            # TODO
            batch_x.append(x)
            batch_y.append(y)
            lengths_x.append(len(x))
            lengths_y.append(len(y))

        # pack the mfccs and transcripts using the pad_sequence function from pytorch
        batch_x_pad = pad_sequence(batch_x, batch_first=True, padding_value=PAD_TOKEN)# TODO
        batch_y_pad = pad_sequence(batch_y, batch_first=True, padding_value=PAD_TOKEN)# TODO

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)

In [40]:
# Silence the logs because otherwise its just too much

mne.set_log_level(False)

def bandpass_filter_and_stack(mne_raw, all = True):
  if not all:
    theta = mne_raw.copy().filter(4, 8).get_data().T
    alpha = mne_raw.copy().filter(8, 12).get_data().T
    beta = mne_raw.copy().filter(12, 20).get_data().T
    gamma = mne_raw.copy().filter(20, 40).get_data().T

    stacked = np.stack([theta, alpha, beta, gamma], axis = 1)
    return stacked
  else:
    all = mne_raw.copy().filter(4, 8).get_data().T
    return all



In [41]:
len(train_targets)

451

In [42]:
train_data = EEGSpeechDatasetME(EEG_data = [bandpass_filter_and_stack(i) for i in train_eeg], # .get_data().T for i in range(0, 4)],
                     transcripts = train_targets)

val_data = EEGSpeechDatasetME(EEG_data = [bandpass_filter_and_stack(i) for i in val_eeg], # [extracted_sentences[4].get_data().T],
                     transcripts = val_targets)


# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 2, # <--- probably want to increase this :)
    batch_size  = config["batch_size"],
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config["batch_size"],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = train_data.collate_fn
)

# test_loader = torch.utils.data.DataLoader(
#     dataset     = test_data,
#     num_workers = 2,
#     batch_size  = 2,  # not a lot of samples . . .
#     pin_memory  = True,
#     shuffle     = False,
#     collate_fn  = train_data.collate_fn
# )

In [43]:
train_data.EEG_files[0].shape

(5738, 62)

In [44]:
if True: #set to true if you want to do this 30 minute step. did it once and it worked
  def verify_dataset(dataset, partition= 'train-clean-100'):
      print("Max mfcc length          : ", np.max([data[0].shape[0] for data in dataset]))
      print("Avg mfcc length          : ", np.mean([data[0].shape[0] for data in dataset]))
      print("Max transcript length    : ", np.max([data[1].shape[0] for data in dataset]))
      print("Max transcript length    : ", np.mean([data[1].shape[0] for data in dataset]))

  verify_dataset(train_data)
  verify_dataset(val_data)
  dataset_max_len  = max(
      np.max([data[0].shape[0] for data in train_data]),
      np.max([data[0].shape[0] for data in val_data]),
  )
  print("\nMax Length: ", dataset_max_len)

Max mfcc length          :  7005
Avg mfcc length          :  6068.569844789357
Max transcript length    :  1052
Max transcript length    :  868.19955654102
Max mfcc length          :  7005
Avg mfcc length          :  6099.380530973452
Max transcript length    :  1052
Max transcript length    :  873.0265486725664

Max Length:  7005


In [45]:
# # sanity check
# print("Batch size: ", config["batch_size"])
# print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))

# for i, data in enumerate(train_loader):
#     x, y, lx, ly = data
#     print(x.shape, y.shape, lx.shape, ly.shape)
#     print(y)
#     break

In [46]:
class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

def plot_attention(attention):
    # Function for plotting attention
    # You need to get a diagonal plot
    plt.clf()
    seaborn.heatmap(attention, cmap='GnBu')
    plt.show()

def save_model(model, optimizer, scheduler, tf_scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         'tf_rate'                  : tf_rate,
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(best_path, epoch_path, model, mode= 'best', metric= 'valid_acc', optimizer= None, scheduler= None, tf_scheduler= None):


    if mode == 'best':
        checkpoint  = torch.load(best_path)
        print("Loading best checkpoint: ", checkpoint[metric])
    else:
        checkpoint  = torch.load(epoch_path)
        print("Loading epoch checkpoint: ", checkpoint[metric])

    model.load_state_dict(checkpoint['model_state_dict'], strict= False)

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        #optimizer.param_groups[0]['lr'] = 1.5e-3
        optimizer.param_groups[0]['weight_decay'] = 1e-5
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    if tf_scheduler != None:
        tf_scheduler    = checkpoint['tf_scheduler']

    epoch   = checkpoint['epoch']
    metric  = torch.load(best_path)[metric]

    return [model, optimizer, scheduler, tf_scheduler, epoch, metric]

class TimeElapsed():
    def __init__(self):
        self.start  = -1

    def time_elapsed(self):
        if self.start == -1:
            self.start = time.time()
        else:
            end = time.time() - self.start
            hrs, rem    = divmod(end, 3600)
            min, sec    = divmod(rem, 60)
            min         = min + 60*hrs
            print("Time Elapsed: {:0>2}:{:02}".format(int(min),int(sec)))
            self.start  = -1

In [47]:
import math

class PositionalEncoding(torch.nn.Module):

    def __init__(self, projection_size, max_seq_len= 3500, dropout=0.2):
        super().__init__()
        # Read the Attention Is All You Need paper to learn how to code code the positional encoding

        # TODO
        self.dropout = nn.Dropout(dropout)
        K = max_seq_len     #???
        d = projection_size #???

        #self.P = torch.zeros((K,d)).to(DEVICE) #K is position of object in sequence, d is dimension of output embedding

        #for t in range(0, max_seq_len):
        #  for i in range(0, int(d/2)):
        #    denominator = 10000 ** (2*i/d)
        #    self.P[t,2*i] = math.sin(t/denominator)
        #    self.P[t,2*i+1] = math.cos(t/denominator)
        #self.P = self.P.unsqueeze(0)

        self.P = torch.zeros(max_seq_len, projection_size).to(DEVICE)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, projection_size, 2).float() * (-torch.log(torch.tensor(10000.0)) / projection_size))
        self.P[:, 0::2] = torch.sin(position * div_term)
        self.P[:, 1::2] = torch.cos(position * div_term)
        self.P = self.P.unsqueeze(0)


    def forward(self, x):
        #x = x + self.P[:, :x.shape[0],:]
        #print('forward')
        #print(x.shape)
        #print(self.P.shape)
        x = x + self.P[:, :x.size(1), :]
        #print(x.shape)
        x = self.dropout(x)
        return x

class TransformerEncoder(torch.nn.Module):
    def __init__(self, projection_size, num_heads, dropout= 0.2):
        super().__init__()

        # create the key, query and value weights
        #print(projection_size)
        self.KW         = nn.Linear(projection_size, projection_size) # TODO
        self.VW         = nn.Linear(projection_size, projection_size) # TODO
        self.QW         = nn.Linear(projection_size, projection_size) # TODO

        self.permute    = PermuteBlock()
        # Compute multihead attention. You are free to use the version provided by pytorch
        self.attention  = torch.nn.MultiheadAttention(embed_dim=projection_size, num_heads=num_heads, dropout=dropout, batch_first=True) # TODO

        #ohhh bn means batch norm
        self.bn1        = nn.LayerNorm(projection_size, eps=1e-6) # TODO

        self.bn2        = nn.LayerNorm(projection_size, eps=1e-6) # TODO

        # Feed forward neural network
        self.MLP        = nn.Linear(projection_size,projection_size) #???
        self.activ      = nn.ReLU()
        self.MLP2        = nn.Linear(projection_size,projection_size)

    def forward(self, x):
        # compute the key, query and value
        #print(self.projection_size.shape)
        #print(x.shape)
        #if len(x.shape) > 2:
          #x = torch.permute(x, (1,2,0))
        #print('transformer1')
        x = x.to(DEVICE)
        #print(x.shape)
        key     = self.KW.forward(x) # TODO
        #print('transformer2')
        value   = self.VW.forward(x) # TODO
        query   = self.QW.forward(x) # TODO
        #print('transformer2')

        # compute the output of the attention module
        out1    = self.attention.forward(query, key, value, need_weights=False) # TODO
        #print('transformer3')
        # Create a residual connection between the input and the output of the attention module
        #print('hi ', out1.shape)
        #print(x.shape)
        #x = torch.permute(x,(2,0,1))
        out1    = out1[0] + x
        # Apply batch norm to out1
        out1    = self.bn1(out1)

        # Apply the output of the feed forward network
        out2    = self.MLP(out1) # TODO
        out2    = self.activ(out2)
        out2    = self.MLP2(out2)
        # Apply a residual connection between the input and output of the  FFN
        out2    = out2 + out1 # TODO
        # Apply batch norm to the output
        out3    = self.bn2(out2) # TODO

        return out3

model   = TransformerEncoder(
    projection_size  = 256,
    num_heads=4
).to(DEVICE)


print(model)

x_sample    = torch.rand(128, 176, 256)
torchsummaryX.summary(model, x_sample.to(DEVICE))
del x_sample

TransformerEncoder(
  (KW): Linear(in_features=256, out_features=256, bias=True)
  (VW): Linear(in_features=256, out_features=256, bias=True)
  (QW): Linear(in_features=256, out_features=256, bias=True)
  (permute): PermuteBlock()
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (bn1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
  (bn2): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
  (MLP): Linear(in_features=256, out_features=256, bias=True)
  (activ): ReLU()
  (MLP2): Linear(in_features=256, out_features=256, bias=True)
)
        Kernel Shape     Output Shape   Params Mult-Adds
Layer                                                   
0_bn1          [256]  [128, 176, 256]    512.0     256.0
1_MLP     [256, 256]  [128, 176, 256]  65.792k   65.536k
2_activ            -  [128, 176, 256]        -         -
3_MLP2    [256, 256]  [128, 176, 256]  65.792k   65.536k
4_bn2          [256]  [

/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


In [48]:
class TransformerListener(torch.nn.Module):

    def __init__(self,
                 input_size              = 62,
                 base_lstm_layers        = 1,
                 pblstm_layers           = 1,
                 listener_hidden_size    = 512,
                 n_heads                 = 8,
                 tf_blocks               = 1):
        super().__init__()

        # create an lstm layer
        self.base_lstm      = nn.LSTM(input_size = input_size, hidden_size = listener_hidden_size, num_layers = base_lstm_layers, bidirectional=True, batch_first=True)


        # create a sequence of Conv1d layers
        self.kernel_size = 3
        self.embedding      = torch.nn.Conv1d(1024, 512, self.kernel_size, 3)
        #elf.embedding      = torch.nn.Embedding(num_embeddings= len(VOCAB), embedding_dim = 1024, padding_idx= PAD_TOKEN)

        # compute the position encoding
        self.positional_encoding    = PositionalEncoding(projection_size=512) # TODO

        # create a sequence of transformer blocks
        #self.transformer_encoder    = torch.nn.Sequential(nn.Linear(128, 128), nn.Linear(128, 128))
        self.transformer_encoder = TransformerEncoder(projection_size=512, num_heads=n_heads)
        #for i in range(tf_blocks):
        #    pass
        #    # TODO

    def forward(self, x, x_len):

        # pack the inputs before passing them to the LSTm
        #print(x.shape)
        #print(x_len.shape)
        #print('listenerforward1')
        x_packed                = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False) # TODO
        # Pass the packed sequence through the lstm
        #print('hi ', x_packed[0].shape)
        #print('listenerforward2')
        lstm_out, _             = self.base_lstm(x_packed)
        #print('hi')
        # Unpack the output of the lstm
        #print('listenerforward3')
        output, output_lengths = pad_packed_sequence(lstm_out, batch_first=True)

        # Pass the output through the embedding
        #print('listenerforward4')
        output = torch.permute(output,(0,2,1))
        output = self.embedding(output)
        output = torch.permute(output,(0,2,1))
        # calculate the new output length

        output_lengths          = 1 + (output_lengths-self.kernel_size)//2

        # calculate the position encoding
        #print('listenerforward5')
        output  = self.positional_encoding(output)
        #print('encoding')
        #print(pos_enc.shape)
        # Pass the output of the positional encoding through the transformer encoder
        #print('listenerforward6')
        #output  = self.transformer_encoder(output)
        #print('listenerforward7')
        #print('listener')
        #print(x.shape)
        #print(x_len.shape)
        #print(output.shape)
        #print(output_lengths.shape)

        return output, output_lengths

In [49]:
class Attention(torch.nn.Module):
  def __init__(self, listener_hidden_size=512, speller_hidden_size=256, projection_size=256):
    super().__init__()
    self.WK = nn.Linear(listener_hidden_size, projection_size)
    self.WV = nn.Linear(listener_hidden_size, projection_size)
    self.WQ = nn.Linear(speller_hidden_size, projection_size)
    self.softmax = nn.Softmax(dim=1)

  def set_key_value(self, encoder_outputs):
    #for b in range(0, encoder_outputs.shape[0]):
    self.key = self.WK.forward(encoder_outputs)
    self.value = self.WV.forward(encoder_outputs)


  def compute_context(self, decoder_context):
    #print('YO', decoder_context.shape)
    #print(self.key.shape)
    #attention_weights = torch.zeros(config['batch_size'], decoder_context.shape[0], self.key.shape[1])
    #context = torch.zeros(config['batch_size'], self.value.shape[1])
    query_length = len(decoder_context[0,:])

    query = self.WQ(decoder_context)
    self.query = query
    #print('query key weights ', query.shape)
    #print(self.key.shape)
    #raw_weights = (1/math.sqrt(query_length)) * torch.bmm(query.unsqueeze(1), torch.transpose(self.key, 1, 2))
    #print(self.key.shape)
    #print(query.shape)
    raw_weights = (1/math.sqrt(query_length)) * torch.bmm(self.key, query.unsqueeze(2))
    raw_weights = raw_weights.squeeze(2)
    #print(raw_weights.shape)
    attention_weights = self.softmax(raw_weights)
    #print(attention_weights.shape)
    #print('context ', context.shape)
    #print(self.value.shape)
    context = torch.bmm(attention_weights.unsqueeze(1), self.value).squeeze(1)

    return context, attention_weights

In [50]:
from sre_constants import JUMP
class Speller(torch.nn.Module):

  # Refer to your HW4P1 implementation for help with setting up the language model.
  # The only thing you need to implement on top of your HW4P1 model is the attention module and teacher forcing.

  def __init__(self, attender:Attention):
    super(). __init__()

    self.attend = attender # Attention object in speller
    #self.max_timesteps = 600 # Max transcript length
    self.max_timesteps = 1100

    #self.embedding = torch.nn.Conv1d(28, 64, 3, 2) # Embedding layer to convert token to latent space
    self.embedding = nn.Embedding(num_embeddings= len(VOCAB), embedding_dim = 512, padding_idx= PAD_TOKEN) #embed to lower dimension ??
    self.positional_encoding    = PositionalEncoding(projection_size=512)

    self.lstm_output_chan = 256
    self.lstm_cells = nn.ModuleList([nn.LSTMCell(768, 512),  nn.LSTMCell(512, self.lstm_output_chan)]) # Create a sequence of LSTM Cells
    self.lstm_cells = nn.Sequential(*self.lstm_cells)
    #self.lstm_cells = nn.LSTMCell(71,self.lstm_output_chan)
    drop_layers    = [nn.Dropout(p=0.2)] * len(self.lstm_cells)
    self.dropout_lays = nn.ModuleList(drop_layers)
    # For CDN (Feel free to change)
    self.output_to_char = nn.Linear(512, len(VOCAB)) # Linear module to convert outputs to correct hidden size (Optional: TO make dimensions match)
    #self.activation = nn.Tanh() # Check which activation is suggested
    #self.char_prob = nn.Softmax(dim=1) # Linear layer to convert hidden space back to logits for token classification
    self.output_to_char.weight = self.embedding.weight # Weight tying (From embedding layer)


  def lstm_step(self, input_word, hidden_state):
    #print(lstm_input)
    #print(hidden_state)
    #for j in range(len(self.lstm_cells)):
    #    # Feed the input through each LSTM Cell
    #    print(lstm_input.dim())
    #    hidden_state[j] = self.lstm_cells.forward(lstm_input)
    #    lstm_input = hidden_state[j]
    #print(lstm_input.shape)
    #print(hidden_state.shape)
    #print(input_word.shape)
    for j in range(len(self.lstm_cells)):
        #print(j)
        #print("hidden: ", hidden_state[j] if hidden_state[j] is not None else "None")
        if hidden_state[j] is None:
            hidden_state[j] = self.lstm_cells[j](input_word, hidden_state[j])
        else:
            hidden_state[j] = self.lstm_cells[j](input_word, hidden_state[j])
        #print('check')
        input_word = hidden_state[j][0]
        input_word = self.dropout_lays[j](input_word)
        hidden_state[j] = (input_word, hidden_state[j][1])
    return input_word, hidden_state # What information does forward() need?

  def CDN(self,CDN_input):
    # Make the CDN here, you can add the output-to-char
    out = self.output_to_char.forward(CDN_input)
    #out = self.activation.forward(out)
    #print('look here', out.shape)
    #out = self.char_prob.forward(out)
    #print(out.shape)
    return out

  def forward (self, y=None, teacher_forcing_ratio=1):

    attn_context = torch.zeros((config['batch_size'],256)) # initial context tensor for time t = 0
    output_symbol = torch.tensor(SOS_TOKEN) # Set it to SOS for time t = 0
    output_symbol = output_symbol.repeat((config['batch_size']))
    raw_outputs = []
    attention_plot = []

    if y is None:
        timesteps = self.max_timesteps
        teacher_forcing_ratio = 0 #Why does it become zero? we don't have label y

    else:
        timesteps = len(y[0,:]) # How many timesteps are we predicting for?
        label = self.embedding(y)

    hidden_states_list = [None]*len(self.lstm_cells)
    #hidden_states_list = torch.zeros(config['batch_size'],self.lstm_output_chan)
    # Initialize your hidden_states list here similar to HW4P1

    for t in range(timesteps):
        p = np.random.random_sample() # generate a probability p between 0 and 1

        if p < teacher_forcing_ratio and t > 0: # Why do we consider cases only when t > 0? What is considered when t == 0? Think.
            char_embed = label[:,t-1,:] # Take from y, else draw from probability distribution
        else:
            char_embed = self.embedding(output_symbol.to(DEVICE)) #output symbol remains as output symbol from before

        char_embed = self.positional_encoding(char_embed.unsqueeze(1)).squeeze(1)
        #print(output_symbol)
        #print(output_symbol.shape)
        #print(y.shape)
        output_symbol = output_symbol.to(DEVICE)
        #char_embed = self.embedding(output_symbol) # Embed the character symbol

        #print(y[t-1])
        # Concatenate the character embedding and context from attention, as shown in the diagram
        #print('embedding...')
        #print(char_embed.shape)
        #print(attn_context.shape)
        #print('char ', char_embed.shape)
        #print(attn_context.shape)
        lstm_input = torch.cat((char_embed, attn_context.to(DEVICE)),1)

        input_word, hidden_states_list = self.lstm_step(lstm_input, hidden_states_list) # Feed the input through LSTM Cells and attention.
        # What should we retrieve from forward_step to prepare for the next timestep?
        attn_context, attn_weights = self.attend.compute_context(hidden_states_list[-1][0]) # Feed the resulting hidden state into attention

        #print(self.attend.query.shape)
        #print(attn_context.shape)
        cdn_input = torch.cat((self.attend.query, attn_context.to(DEVICE)),1) # TODO: You need to concatenate the context from the attention module with the LSTM output hidden state, as shown in the diagram

        raw_pred = self.CDN(cdn_input) # call CDN with cdn_input

        # Generate a prediction for this timestep and collect it in output_symbols
        #print(raw_pred)
        #print(raw_pred.shape)
        #print(raw_pred)
        #print(raw_pred.shape)
        output_symbol = torch.argmax(raw_pred,1) # Draw correctly from raw_pred

        raw_outputs.append(raw_pred) # for loss calculation
        attention_plot.append(attn_weights) # for plotting attention plot


    attention_plot = torch.stack(attention_plot, dim=1)
    raw_outputs = torch.stack(raw_outputs, dim=1)

    return raw_outputs, attention_plot

In [51]:
class ASRModel(torch.nn.Module):
  def __init__(self): # add parameters
    super().__init__()

    # Pass the right parameters here
    self.listener = TransformerListener()
    self.attend = Attention()
    self.speller = Speller(self.attend)

  def forward(self, x,lx,y=None,teacher_forcing_ratio=1):
    # Encode speech features
    encoder_outputs, lx = self.listener(x,lx)

    # We want to compute keys and values ahead of the decoding step, as they are constant for all timesteps
    # Set keys and values using the encoder outputs
    #print(encoder_outputs.shape)
    self.attend.set_key_value(encoder_outputs)

    # Decode text with the speller using context from the attention
    raw_outputs, attention_plots = self.speller(y=y,teacher_forcing_ratio=teacher_forcing_ratio)

    return raw_outputs, attention_plots

In [52]:
torch.cuda.empty_cache()
gc.collect()

1348

In [53]:
model = ASRModel(

    # Initialize your model
)

model = model.to(DEVICE)
print(model)

x_sample    = torch.rand(36, 1700, 62)
lx_sample   = torch.randint(1, 500, (36,))
torchsummaryX.summary(model, x_sample.to(DEVICE), lx_sample)
del x_sample
del lx_sample

ASRModel(
  (listener): TransformerListener(
    (base_lstm): LSTM(62, 512, batch_first=True, bidirectional=True)
    (embedding): Conv1d(1024, 512, kernel_size=(3,), stride=(3,))
    (positional_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (transformer_encoder): TransformerEncoder(
      (KW): Linear(in_features=512, out_features=512, bias=True)
      (VW): Linear(in_features=512, out_features=512, bias=True)
      (QW): Linear(in_features=512, out_features=512, bias=True)
      (permute): PermuteBlock()
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
      )
      (bn1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (bn2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (MLP): Linear(in_features=512, out_features=512, bias=True)
      (activ): ReLU()
      (MLP2): Linear(in_features=512, out_features=512, bias=True)
    )
  )
  

/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


                                                  Kernel Shape  \
Layer                                                            
0_listener.LSTM_base_lstm                                    -   
1_listener.Conv1d_embedding                     [1024, 512, 3]   
2_listener.positional_encoding.Dropout_dropout               -   
3_speller.Embedding_embedding                        [512, 31]   
4_speller.positional_encoding.Dropout_dropout                -   
...                                                        ...   
10998_speller.dropout_lays.Dropout_0                         -   
10999_speller.attend.Linear_WQ                      [256, 256]   
11000_speller.attend.Linear_WQ                      [256, 256]   
11001_speller.attend.Softmax_softmax                         -   
11002_speller.attend.Softmax_softmax                         -   

                                                  Output Shape     Params  \
Layer                                                           

In [54]:
optimizer   = torch.optim.Adam(model.parameters(), lr=config["learning_rate"], amsgrad=True, weight_decay=0) # TODO

criterion   = torch.nn.CrossEntropyLoss(ignore_index = 0) # TODO

scaler      = torch.cuda.amp.GradScaler()

scheduler   = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.95, last_epoch=-1)

In [55]:
# We have given you this utility function which takes a sequence of indices and converts them to a list of characters
def indices_to_chars(indices, vocab):
    tokens = []
    for i in indices: # This loops through all the indices
        if int(i) == SOS_TOKEN: # If SOS is encountered, dont add it to the final list
            continue
        elif int(i) == EOS_TOKEN: # If EOS is encountered, stop the decoding process
            break
        else:
            tokens.append(vocab[i])
    return tokens

# To make your life more easier, we have given the Levenshtein distantce / Edit distance calculation code
def calc_edit_distance(predictions, y, y_len, vocab= VOCAB, print_example= False, batch_num = -1):

    dist                = 0
    batch_size, seq_len = predictions.shape

    for batch_idx in range(batch_size):

        y_sliced    = indices_to_chars(y[batch_idx,0:y_len[batch_idx]], vocab)
        pred_sliced = indices_to_chars(predictions[batch_idx], vocab)

        # Strings - When you are using characters from the AudioDataset
        y_string    = ''.join(y_sliced)
        pred_string = ''.join(pred_sliced)

        #dist        += Levenshtein.distance(pred_string, y_string)
        # Comment the above abd uncomment below for toy dataset
        dist      += Levenshtein.distance(y_sliced, pred_sliced)

    if (print_example) and (batch_num == 0):
        # Print y_sliced and pred_sliced if you are using the toy dataset
        print("\nGround Truth : ", y_string)
        print("Prediction   : ", pred_string)

    dist    /= batch_size
    return dist


def train(model, dataloader, criterion, optimizer, teacher_forcing_rate):

    model.train()
    batch_bar = blue_tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    running_loss        = 0.0
    running_perplexity  = 0.0

    for i, (x, y, lx, ly) in enumerate(dataloader):
        if i == 12:
          continue
        optimizer.zero_grad()

        x, y, lx, ly = x.to(device), y.to(device), lx, ly

        with torch.cuda.amp.autocast():

            raw_predictions, attention_plot = model(x, lx, y= y, teacher_forcing_ratio = teacher_forcing_rate)

            # Predictions are of Shape (batch_size, timesteps, vocab_size).
            # Transcripts are of shape (batch_size, timesteps) Which means that you have batch_size amount of batches with timestep number of tokens.
            # So in total, you have batch_size*timesteps amount of characters.
            # Similarly, in predictions, you have batch_size*timesteps amount of probability distributions.
            # How do you need to modify transcipts and predictions so that you can calculate the CrossEntropyLoss? Hint: Use Reshape/View and read the docs
            # Also we recommend you plot the attention weights, you should get convergence in around 10 epochs, if not, there could be something wrong with
            # your implementation

            # loss function has been set to ingnore padding index
            # print("Pred shape: ", raw_predictions.shape, raw_predictions.data.shape)
            # print("Y shape: ", y.shape, y.data.shape)

            loss        =  criterion(raw_predictions.view(-1, raw_predictions.shape[2]), y.view(-1,)) # TODO: Cross Entropy Loss

            perplexity  = torch.exp(loss) # Perplexity is defined the exponential of the loss
            running_loss        += loss.item()
            running_perplexity  += perplexity.item()

        # Backward on the masked loss
        scaler.scale(loss).backward()
        # loss.backward()

        # Optional: Use torch.nn.utils.clip_grad_norm to clip gradients to prevent them from exploding, if necessary
        # If using with mixed precision, unscale the Optimizer First before doing gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10)

        scaler.step(optimizer)
        scaler.update()


        batch_bar.set_postfix(
            loss="{:.04f}".format(running_loss/(i+1)),
            perplexity="{:.04f}".format(running_perplexity/(i+1)),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
            tf_rate='{:.02f}'.format(teacher_forcing_rate))
        batch_bar.update()

        del x, y, lx, ly
        torch.cuda.empty_cache()

    running_loss /= len(dataloader)
    running_perplexity /= len(dataloader)
    batch_bar.close()

    return running_loss, running_perplexity, attention_plot

def validate(model, dataloader):

    model.eval()

    batch_bar = blue_tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc="Val")

    running_lev_dist = 0.0

    for i, (x, y, lx, ly) in enumerate(dataloader):
        if i == 3:
          continue
        x, y, lx, ly = x.to(device), y.to(device), lx, ly

        with torch.inference_mode():
            raw_predictions, attentions = model(x, lx, y = None)

        # Greedy Decoding
        greedy_predictions   = torch.argmax(raw_predictions, dim = 2) # TODO: How do you get the most likely character from each distribution in the batch?

        # Calculate Levenshtein Distance
        running_lev_dist    += calc_edit_distance(greedy_predictions, y, ly, VOCAB, print_example = True,
                                                  batch_num = i) # You can use print_example = True for one specific index i in your batches if you want

        batch_bar.set_postfix(
            dist="{:.04f}".format(running_lev_dist/(i+1)))
        batch_bar.update()

        del x, y, lx, ly
        torch.cuda.empty_cache()

    batch_bar.close()
    running_lev_dist /= len(dataloader)

    return running_lev_dist

EEG to phonemes

input EEG --> output phoneme

EEG = (12 * N_subjects,) varying length

"labels" = list (12 * N_subjects,) varying length


Follow HW3P2 logic

Padpacked sequence whatever dataloader

In [56]:
# Login to Wandb
import wandb
wandb.login(key="eb0a91ea57d10554fba242aafcda65f03bb4ccc3")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [57]:
run = wandb.init(
    name = "Attention-Model-Beta", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "IDL_group_project", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

lr,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_perplex,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_dist,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,0.0
train_loss,2.61488
train_perplex,15.68824
valid_dist,573.4375


In [58]:
import seaborn as sns

def plot_attention(attention):
    # Function for plotting attention
    # You need to get a diagonal plot
    plt.clf()
    sns.heatmap(attention, cmap='GnBu')
    plt.show()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

best_lev_dist = float("inf")
tf_rate = 1.0
path = r"./checkpoint.pth"
tf_scheduler = None

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    # Call train and validate, get attention weights from training
    running_loss, running_perplexity, attention_plot = train(model, train_loader, criterion, optimizer, tf_rate)

    # Print your metrics
    print('[TRAIN] \t Loss: %.4f \t Perplexity: %.4f \tLr: %.6f'
          % (running_loss, running_perplexity, optimizer.param_groups[0]['lr']))


    running_lev_dist = validate(model, val_loader)
    print('[VAL] \tLoss: %.4f'% (running_lev_dist))

    # Plot Attention for a single item in the batch
    plot_attention(attention_plot[0].detach().cpu())

    # Log metrics to Wandb
    wandb.log({
        'train_loss': running_loss,
        'train_perplex': running_perplexity,
        'valid_dist': running_lev_dist,
        'lr'        : optimizer.param_groups[0]['lr'],
    })

    # Optional: Scheduler Step / Teacher Force Schedule Step
    scheduler.step(running_lev_dist)

    if running_lev_dist <= best_lev_dist:
        best_lev_dist = running_lev_dist
        metric = ["valid_dist", running_lev_dist]
        save_model(model, optimizer, scheduler, tf_scheduler, metric, epoch, path)
        wandb.save(os.path.join(wandb.run.dir, "checkpoint*"))
        print("Saved best model")
        # Save your model checkpoint here

Output hidden; open in https://colab.research.google.com to view.